In [1]:
import os

from OCC.Core.IFSelect import IFSelect_RetDone
from OCC.Core.STEPCAFControl import STEPCAFControl_Reader
from OCC.Core.TCollection import TCollection_ExtendedString
from OCC.Core.TDocStd import TDocStd_Document
from OCC.Core.VrmlAPI import VrmlAPI_Writer
from OCC.Core.XCAFDoc import XCAFDoc_ShapeTool, XCAFDoc_DocumentTool
from OCC.Core.BRep import BRep_Tool
from OCC.Core.TDF import TDF_LabelSequence, TDF_Label
from OCC.Extend.TopologyUtils import TopologyExplorer
from OCC.Core.TopAbs import topabs
from OCC.Extend.DataExchange import read_step_file_with_names_colors
from OCC.Display.WebGl.x3dom_renderer import X3DomRenderer, X3DExporter
from IPython.display import HTML
import xml.etree.ElementTree as ET
import re

In [2]:
#stp_filename = 'VentilatorAP203' #'as1_pe_203'
stp_filename = 'as1_pe_203'
stp_path = os.path.join('..','assets', 'step', stp_filename + '.stp')
#stp_path = os.path.join(stp_filename + '.stp')
stpShapes = read_step_file_with_names_colors(stp_path)


Number of shapes at root : 1

Name : AS1_PE_ASM
Name : PLATE
Name : SOLID
    instance color Name & RGB:  <class 'Quantity_Color'> 229 0.0 1.0 0.0
Name : COMPOUND
    shape color Name & RGB:  <class 'Quantity_Color'> 510 1.0 1.0 0.0
    shape color Name & RGB:  <class 'Quantity_Color'> 510 1.0 1.0 0.0
    shape color Name & RGB:  <class 'Quantity_Color'> 510 1.0 1.0 0.0
    shape color Name & RGB:  <class 'Quantity_Color'> 510 1.0 1.0 0.0
    shape color Name & RGB:  <class 'Quantity_Color'> 510 1.0 1.0 0.0
    shape color Name & RGB:  <class 'Quantity_Color'> 510 1.0 1.0 0.0
    shape color Name & RGB:  <class 'Quantity_Color'> 510 1.0 1.0 0.0
Name : L_BRACKET_ASSEMBLY_ASM
Name : L-BRACKET
Name : SOLID
    instance color Name & RGB:  <class 'Quantity_Color'> 510 1.0 1.0 0.0
Name : COMPOUND
    shape color Name & RGB:  <class 'Quantity_Color'> 510 1.0 1.0 0.0
    shape color Name & RGB:  <class 'Quantity_Color'> 510 1.0 1.0 0.0
    shape color Name & RGB:  <class 'Quantity_Color'> 510

In [13]:
x3domRenderer = X3DomRenderer(path="./")
for shape in stpShapes:
    #print(shape.ShapeType(),stpShapes[shape])
    #x3dexp = X3DExporter(shape, None, None, False, (1,0,0), (0,0,0), 0, 0, (0,0,0), 2, 1)
    #x3dexp.compute()
    #print(len(x3dexp._triangle_sets), len(x3dexp._line_sets))
    color = stpShapes[shape][1]
    colorRGB=(color.Red(), color.Green(), color.Blue())
    x3domRenderer.DisplayShape(shape=shape, color=colorRGB, export_edges=True)
x3domRenderer.generate_html_file(False, 1.0)
    
    


## x3dom webgl renderer - render axes/planes : True - axes/plane zoom factor : 1
X3D exporter, discretize an edge
X3D exporter, discretize an edge
X3D exporter, discretize an edge
X3D exporter, discretize an edge
X3D exporter, discretize an edge
X3D exporter, discretize an edge
X3D exporter, discretize an edge
X3D exporter, discretize an edge
X3D exporter, discretize an edge
X3D exporter, discretize an edge
X3D exporter, discretize an edge
X3D exporter, discretize an edge
X3D exporter, discretize an edge
X3D exporter, discretize an edge
X3D exporter, discretize an edge
X3D exporter, discretize an edge
X3D exporter, discretize an edge
X3D exporter, discretize an edge
X3D exporter, discretize an edge
X3D exporter, discretize an edge
X3D exporter, discretize an edge
X3D exporter, discretize an edge
X3D exporter, discretize an edge
X3D exporter, discretize an edge
X3D exporter, discretize an edge
X3D exporter, discretize an edge
X3D exporter, discretize an edge
X3D exporter, discretize an 

In [27]:
#merge shp linesets
for hash in x3domRenderer._x3d_shapes.keys():
    xml=ET.parse(hash+'.x3d')
    #print(xml)
    shp_vertexCounts=[]
    shp_points=[]
    for lineset in xml.iter('LineSet'):
        shp_vertexCounts.append(lineset.get('vertexCount'))
        shp_points.append(list(lineset.iter('Coordinate'))[0].get('point'))
    #print (shp_vertexCounts, shp_points)
    #reuse first lineset to hold all ines
    lset=list(xml.iter('LineSet'))[0]
    lset.set('vertexCount',",".join(shp_vertexCounts))
    coords=list(lset.iter('Coordinate'))[0]
    coords.set('point',",".join(shp_points))
    #remove all other lineset shapes
    group=list(xml.iter('Group'))[0]
    transforms=list(group.iter('Transform'))
    #skip first
    for transform in transforms[1:]:
        group.remove(transform)
    #print(ET.dump(group))
    xml.write("M_"+hash+'.x3d')

In [28]:
#merge edg linesets
edg_vertexCounts=[]
edg_points=[]
for hash in x3domRenderer._x3d_edges.keys():
    xml=ET.parse(hash+'.x3d').getroot()
    firstlineset=list(xml.iter('LineSet'))[0]
    edg_vertexCounts.append(firstlineset.get('vertexCount'))
    edg_points.append(list(firstlineset.iter('Coordinate'))[0].get('point'))

In [30]:
firstedgex3d=list(x3domRenderer._x3d_edges.keys())[0]+'.x3d'
xml=ET.parse(hash+'.x3d')
lset=list(xml.iter('LineSet'))[0]
lset.set('vertexCount',",".join(edg_vertexCounts))
coords=list(lset.iter('Coordinate'))[0]
coords.set('point',",".join(edg_points))
ALLEDGESX3D='allEdges.x3d'
xml.write(ALLEDGESX3D)

In [31]:
with open('index.html','r') as file:
    html = file.read()

In [32]:
html2=html.replace(firstedgex3d, ALLEDGESX3D)
html2=re.sub(r"<Inline.*edg.*</Inline>","", html2)
html2=re.sub(r'url="shp','url="M_shp', html2)
print(html2)

<!DOCTYPE HTML>
<html lang="en">
<head>
    <title>pythonOCC 7.4.0 x3dom renderer</title>
    <meta name='Author' content='Thomas Paviot - tpaviot@gmail.com'>
    <meta name='Keywords' content='WebGl,pythonOCC'>
    <meta charset="utf-8">
    <link rel="stylesheet" type="text/css" href="https://x3dom.org/release/x3dom.css">
    <script src="https://x3dom.org/release/x3dom.js"></script>
    <style>
        body {
            background: linear-gradient(#ced7de, #808080);
            margin: 0px;
            overflow: hidden;
        }
        #pythonocc_rocks {
            padding: 5px;
            position: absolute;
            left: 1%;
            bottom: 2%;
            height: 38px;
            width: 280px;
            border-radius: 5px;
            border: 2px solid #f7941e;
            opacity: 0.7;
            font-family: Arial;
            background-color: #414042;
            color: #ffffff;
            font-size: 14px;
            opacity: 0.5;
        }
        #command

In [33]:
HTML(html2
     .replace('event.preventDefault','//event.preventDefault')
     .replace('height: 38px','_height: 38px')
     .replace('height: 65px','_height: 65px') )